In [ ]:
from astropy.io import fits
import pandas as pd
from astropy.table import Table
from astropy.table import vstack
from matplotlib import pyplot as plt
import numpy as np
from numcosmo_py import Ncm, Nc
from numcosmo_py.external.pyssc import pyssc as PySSC
Ncm.cfg_init()

In [ ]:

catalog_halos_00_4 = Table.read('halos/z0p00_hlist_4.fits', format='fits')
catalog_halos_00_99 = Table.read('halos/z0p00_hlist_99.fits', format='fits')
catalog_halos_00 = vstack([catalog_halos_00_4 , catalog_halos_00_99])


catalog_halos_02_4 = Table.read('halos/z0p02_hlist_4.fits', format='fits')
catalog_halos_02_95 = Table.read('halos/z0p02_hlist_95.fits', format='fits')
catalog_halos_02_99 = Table.read('halos/z0p02_hlist_99.fits', format='fits')
catalog_halos_02 = vstack([catalog_halos_02_4 , catalog_halos_02_95 , catalog_halos_02_99])

catalog_halos_05_4 = Table.read('halos/z0p05_hlist_4.fits', format='fits')
catalog_halos_05_89 = Table.read('halos/z0p05_hlist_89.fits', format='fits')
catalog_halos_05_90 = Table.read('halos/z0p05_hlist_90.fits', format='fits')
catalog_halos_05_94 = Table.read('halos/z0p05_hlist_94.fits', format='fits')
catalog_halos_05_95 = Table.read('halos/z0p05_hlist_95.fits', format='fits')
catalog_halos_05_99 = Table.read('halos/z0p05_hlist_99.fits', format='fits')
catalog_halos_05 = vstack([catalog_halos_05_4 , catalog_halos_05_89 ,catalog_halos_05_90 ,catalog_halos_05_94 , 
                           catalog_halos_05_95 , catalog_halos_05_99])

catalog_halos_09_84 = Table.read('halos/z0p09_hlist_84.fits', format='fits')
catalog_halos_09_89 = Table.read('halos/z0p09_hlist_89.fits', format='fits')
catalog_halos_09_94 = Table.read('halos/z0p09_hlist_94.fits', format='fits')
catalog_halos_09_99 = Table.read('halos/z0p09_hlist_99.fits', format='fits')
catalog_halos_09 = vstack([catalog_halos_09_84 , catalog_halos_09_89 , catalog_halos_09_94 , catalog_halos_09_99])

catalog_halos_14_79 = Table.read('halos/z0p14_hlist_79.fits', format='fits')
catalog_halos_14_84 = Table.read('halos/z0p14_hlist_84.fits', format='fits')
catalog_halos_14_89 = Table.read('halos/z0p14_hlist_89.fits', format='fits')
catalog_halos_14_93 = Table.read('halos/z0p14_hlist_93.fits', format='fits')
catalog_halos_14_94 = Table.read('halos/z0p14_hlist_94.fits', format='fits')
catalog_halos_14_98 = Table.read('halos/z0p14_hlist_98.fits', format='fits')
catalog_halos_14_99 = Table.read('halos/z0p14_hlist_99.fits', format='fits')
catalog_halos_14 = vstack([catalog_halos_14_79 , catalog_halos_14_84 , catalog_halos_14_89 , catalog_halos_14_93 , catalog_halos_14_94 ,
                         catalog_halos_14_98  ,catalog_halos_14_99])

catalog_halos_19_69 = Table.read('halos/z0p19_hlist_69.fits', format='fits')
catalog_halos_19_74 = Table.read('halos/z0p19_hlist_74.fits', format='fits')
catalog_halos_19_78 = Table.read('halos/z0p19_hlist_78.fits', format='fits')
catalog_halos_19_79 = Table.read('halos/z0p19_hlist_79.fits', format='fits')
catalog_halos_19_83 = Table.read('halos/z0p19_hlist_83.fits', format='fits')
catalog_halos_19_84 = Table.read('halos/z0p19_hlist_84.fits', format='fits')
catalog_halos_19_88 = Table.read('halos/z0p19_hlist_88.fits', format='fits')
catalog_halos_19_89 = Table.read('halos/z0p19_hlist_89.fits', format='fits')
catalog_halos_19_93 = Table.read('halos/z0p19_hlist_93.fits', format='fits')
catalog_halos_19_94 = Table.read('halos/z0p19_hlist_94.fits', format='fits')
catalog_halos_19_98 = Table.read('halos/z0p19_hlist_98.fits', format='fits')
catalog_halos_19_99 = Table.read('halos/z0p19_hlist_99.fits', format='fits')
catalog_halos_19 = vstack([catalog_halos_19_74 , catalog_halos_19_79 , catalog_halos_19_83 , catalog_halos_19_84 , catalog_halos_19_88
                          ,catalog_halos_19_89 , catalog_halos_19_93 , catalog_halos_19_94 , catalog_halos_19_98 , catalog_halos_19_99])


catalog_halos = vstack([catalog_halos_00 , catalog_halos_02 , catalog_halos_05 , catalog_halos_09 , catalog_halos_14 , catalog_halos_19])

#catalog_halos_df = catalog_halos.to_pandas()


halos_filtered_0 = catalog_halos[(catalog_halos['pid'] ==  -1)]
halos_filtered = halos_filtered_0[(halos_filtered_0['Mvir'] >= 11) & (halos_filtered_0['Mvir'] <= 14.5) ]
print("oi")
#halos_filtered.write('halos.fits', format='fits')
#display(catalog_halos_00)
#display(catalog_halos_00.columns)

In [ ]:
'''
catalog_halos_25_64 = Table.read('halos/z0p25_hlist_64.fits', format='fits')
catalog_halos_25_69 = Table.read('halos/z0p25_hlist_69.fits', format='fits')
catalog_halos_25_73 = Table.read('halos/z0p25_hlist_73.fits', format='fits')
catalog_halos_25_74 = Table.read('halos/z0p25_hlist_74.fits', format='fits')
catalog_halos_25_78 = Table.read('halos/z0p25_hlist_78.fits', format='fits')
catalog_halos_25_79 = Table.read('halos/z0p25_hlist_79.fits', format='fits')
catalog_halos_25_83 = Table.read('halos/z0p25_hlist_83.fits', format='fits')
catalog_halos_25_84 = Table.read('halos/z0p25_hlist_84.fits', format='fits')
catalog_halos_25_88 = Table.read('halos/z0p25_hlist_88.fits', format='fits')
catalog_halos_25_89 = Table.read('halos/z0p25_hlist_89.fits', format='fits')
catalog_halos_25_93 = Table.read('halos/z0p25_hlist_93.fits', format='fits')
catalog_halos_25_94 = Table.read('halos/z0p25_hlist_94.fits', format='fits')
catalog_halos_25_98 = Table.read('halos/z0p25_hlist_98.fits', format='fits')
catalog_halos_25_99 = Table.read('halos/z0p25_hlist_99.fits', format='fits')
catalog_halos_25 = vstack([catalog_halos_25_64 , catalog_halos_25_69 , catalog_halos_25_73 , catalog_halos_25_74 , catalog_halos_25_78
                           ,catalog_halos_25_79 , catalog_halos_25_83, catalog_halos_25_84, catalog_halos_25_88 , catalog_halos_25_89
                           ,catalog_halos_25_93 , catalog_halos_25_94 , catalog_halos_25_98 , catalog_halos_25_99])
'''

In [ ]:
area = 4109.3
cosmo = Nc.HICosmoDEXcdm()
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("H0", 67.74)
cosmo.param_set_by_name("Omegab", 0.0486)
cosmo.param_set_by_name("Omegac", 0.2603)
cosmo.param_set_by_name("Omegak", 0.00)
cosmo.param_set_by_name("w", -1.0)


prim = Nc.HIPrimPowerLaw.new()
prim.param_set_by_name("ln10e10ASA", 3.02745)
prim.param_set_by_name("n_SA", 0.9667)



reion = Nc.HIReionCamb.new()

cosmo.add_submodel(prim)
cosmo.add_submodel(reion)


dist = Nc.Distance.new(2.0)

tf = Nc.TransferFuncEH()

psml = Nc.PowspecMLTransfer.new(tf)
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()
print("sigma8 = " + str(cosmo.sigma8(psf)))


A_s = np.exp(prim.param_get_by_name("ln10e10ASA")) * 1.0e-10
fact = (0.8159 / psml.sigma_tophat_R(cosmo, 1.0e-7, 0.0, 8.0 / cosmo.h())) ** 2
prim.param_set_by_name("ln10e10ASA", np.log(1.0e10 * A_s * fact))

print("sigma8 = " + str(cosmo.sigma8(psf)))


mulf = Nc.MultiplicityFuncTinker.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200.0)

hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area(area * (np.pi / 180) ** 2)
hmf.prepare(cosmo)
hbias_Tinker = Nc.HaloBiasTinker.new(hmf)
cad = Nc.ClusterAbundance.new(hmf, None)
cad.set_area(area * (np.pi / 180) ** 2)
hmf.prepare(cosmo)

#σ_8 = 0.8159

#mass proxy
cluster_m = Nc.ClusterMassNodist(lnM_min=np.log(10) * (11.000304 - np.log10(cosmo.h())), lnM_max=np.log(10.0) * (14.5  - np.log10(cosmo.h())))

#photo-z
cluster_z = Nc.ClusterRedshiftNodist (z_min = 0.000938 	, z_max = 0.219752)

In [ ]:
multiplicity = [Nc.MultiplicityFuncTinker.new() , Nc.MultiplicityFuncDespali.new() , Nc.MultiplicityFuncDespali.new()] 
legend = ['Tinker','Despali' , 'Despali Virial']

for mult in range(len(multiplicity)):

    mass = np.linspace(np.log(10) * (11.000304 - np.log10(cosmo.h())), np.log(10.0) * (14.5  - np.log10(cosmo.h())), 100)
    mass10 = np.linspace(11.000304 - np.log10(cosmo.h()) , 14.5  - np.log10(cosmo.h()), 100)
    mass_plot = []
    n_bin = []

    mulf = multiplicity[mult]
    mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
    mulf.set_Delta(200.0)
    if mult ==2:
        mulf.set_mdef(Nc.MultiplicityFuncMassDef.VIRIAL)
    
    hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
    hmf.set_area(area * (np.pi / 180) ** 2)
    
    hmf.prepare(cosmo)
    cad = Nc.ClusterAbundance.new(hmf, None)
    hmf.prepare(cosmo)
    cad.prepare(cosmo , cluster_z , cluster_m)
    cad.set_area(area * (np.pi / 180) ** 2)
    print(cad.n(cosmo , cluster_z , cluster_m))
    for i in range(len(mass)-1):
        n_bin.append(cad.intp_bin_d2n(cosmo , cluster_z , cluster_m , [mass[i]] ,[mass[i+1]],None ,[0.000938] , [0.219752] , None))
        mass_plot.append((mass10[i] + mass10[i+1])/2)
    
    
    
    
    
    plt.plot(mass_plot , n_bin , label = "%s" %(legend[mult]))
    plt.xlim(mass_plot[0] , mass_plot[-1])

plt.legend()
plt.xlabel(r"$log_{10}(M/M_{\bigodot})$")
plt.ylabel(r"n_M")
plt.yscale("log")
plt.hist(halos_filtered['Mvir']-np.log10(cosmo.h()), bins=(len(mass10-1)))
plt.savefig("halo_mass_function_log10M.jpg")
plt.show()

In [ ]:
multiplicity = [Nc.MultiplicityFuncTinker.new() , Nc.MultiplicityFuncDespali.new() , Nc.MultiplicityFuncDespali.new()] 
legend = ['Tinker' , 'Despali' , 'Despali Virial']

for mult in range(len(multiplicity)):

    z = np.linspace(0.000938, 0.219752 , 100)
    z_plot = []
    n_bin = []
    
    mulf = multiplicity[mult]
    mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
    mulf.set_Delta(200.0)
    if mult ==2:
        mulf.set_mdef(Nc.MultiplicityFuncMassDef.VIRIAL)

    
    hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
    hmf.set_area(area * (np.pi / 180) ** 2)
    
    hmf.prepare(cosmo)
    cad = Nc.ClusterAbundance.new(hmf, None)
    hmf.prepare(cosmo)
    cad.prepare(cosmo , cluster_z , cluster_m)
    cad.set_area(area * (np.pi / 180) ** 2)
    print(cad.n(cosmo , cluster_z , cluster_m))
    for i in range(len(z)-1):
        n_bin.append(cad.intp_bin_d2n(cosmo , cluster_z , cluster_m , [np.log(10)*(11.000304 - np.log10(cosmo.h()))] ,[np.log(10.0) * (14.5 - np.log10(cosmo.h()))],None ,[z[i]] , [z[i+1]] , None))
        z_plot.append((z[i] + z[i+1])/2)
    
    
    
    
    
    plt.plot(z_plot , n_bin , label = "%s" %(legend[mult]))
    plt.xlim(z_plot[0] , z_plot[-1])

plt.legend()
plt.xlabel(r"$z$")
plt.ylabel(r"n_z")
plt.yscale("log")
plt.hist(halos_filtered['redshift_R'], bins=(len(mass10-1)))
plt.savefig("halo_mass_function_z.jpg")
plt.show()